# Modelos de Difusão com BreastMNIST

Este notebook demonstra como treinar duas redes de difusão não condicionais (**DDPM**) utilizando o dataset **BreastMNIST** do MedMNIST.
O procedimento segue a mesma preparação do notebook `gan_classical_medmnist`, porém substituindo as GANs por modelos de difusão.
Treinaremos um modelo para a classe 0 e outro para a classe 1.


In [ ]:
# Instalação de dependências
!pip install -q diffusers accelerate medmnist datasets torchvision


In [ ]:
# Configuração básica do Accelerate
from accelerate.utils import write_basic_config
write_basic_config()


In [ ]:
# Preparação do dataset BreastMNIST
import os
from pathlib import Path
from medmnist import BreastMNIST
from PIL import Image

data_dir = Path('data/breastmnist')
class0_dir = data_dir / 'class0'
class1_dir = data_dir / 'class1'
class0_dir.mkdir(parents=True, exist_ok=True)
class1_dir.mkdir(parents=True, exist_ok=True)

train_dataset = BreastMNIST(split='train', download=True)
imgs, labels = train_dataset.imgs, train_dataset.labels.flatten()
for idx, (img, label) in enumerate(zip(imgs, labels)):
    if label == 0:
        Image.fromarray(img.squeeze()).save(class0_dir / f'{idx}.png')
    elif label == 1:
        Image.fromarray(img.squeeze()).save(class1_dir / f'{idx}.png')


In [ ]:
# Treinamento dos modelos de difusão para cada classe
!accelerate launch diffusers/examples/unconditional_image_generation/train_unconditional.py --train_data_dir 'data/breastmnist/class0' --resolution 28 --output_dir 'ddpm_breastmnist_class0' --train_batch_size 64 --num_epochs 100 --mixed_precision fp16
!accelerate launch diffusers/examples/unconditional_image_generation/train_unconditional.py --train_data_dir 'data/breastmnist/class1' --resolution 28 --output_dir 'ddpm_breastmnist_class1' --train_batch_size 64 --num_epochs 100 --mixed_precision fp16


In [ ]:
# Geração de imagens a partir dos modelos treinados
from diffusers import DiffusionPipeline
import torch

pipe0 = DiffusionPipeline.from_pretrained('ddpm_breastmnist_class0').to('cuda')
img0 = pipe0().images[0]

pipe1 = DiffusionPipeline.from_pretrained('ddpm_breastmnist_class1').to('cuda')
img1 = pipe1().images[0]
display(img0)
display(img1)
